In [1]:
%run 01-model.ipynb

from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

'' is not a valid SBML 'SId'.


#### Co-factor swapping targets

In [2]:
model.solver = "glpk"

In [3]:
biomass = model.reactions.DRAIN_Biomass

In [4]:
biomass.lower_bound = 0.1 # OBS denne skal måske ikke være 0,1 tjek tutorial notebooken og forstå det


In [5]:
target = model.metabolites.get_by_id("psi")

In [9]:
demand = model.reactions.DM_psi

The demand reaction for the "target" (psilocybin) is defined in "01-model.ipynb"

In [10]:
model.objective = demand

In [11]:
product_yield = product_yield(demand, model.reactions.BOUNDARY_GLCe)

In [12]:
CofactorSwapOptimization?

Init signature:
CofactorSwapOptimization(
    cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)
Docstring:     
Optimize co-factor swapping

As suggested in [1]_, flux through a given reaction can sometimes be optimized by swapping complementary
co-factor. This class implements a search for reactions when swapped improve the given objective. Briefly,
the approach is to

- find reactions that have all the targeted co-factor pairs e.g. (nad_c -> nadp_c, nadh_c -> nadph_c)

- add reactions that have the co-factors swapped and then by a search algorithm switching one off in favor of the
  other

The implementation here differs from that in [1]_ in that we use a general purpose search algorithm rather than
formulating the search as a mixed integer linear programming problem.

References
----------
.. [1] King, Zachary A., and Adam M. Feist. "Optimizing Cofactor Specificity of Oxidoreductase Enzym

In [21]:
swap_opt = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['NAD', 'NADH'], ['NADP', 'NADPH']))

In [22]:
%%time
result = swap_opt.run(max_size=5)

Starting optimization at Mon, 23 Nov 2020 10:04:51


HBox()

Finished after 00:01:24
CPU times: user 2min 25s, sys: 39.5 ms, total: 2min 25s
Wall time: 2min 25s


In [23]:
result

,index,targets,fitness


In [209]:
?cofactor_id_swaps

Type:        tuple
String form: (['NAD', 'NADH'], ['NADP', 'NADPH'])
Length:      2
Docstring:  
tuple() -> empty tuple
tuple(iterable) -> tuple initialized from iterable's items

If the argument is a tuple, the return value is the same object.


In [160]:
?swap_cofactors

Signature: swap_cofactors(reaction, model, swap_pairs, inplace=True)
Docstring:
Swaps the cofactors of a reaction. For speed, it can be done inplace which just changes the coefficients.
If not done inplace, it will create a new Reaction, add it to the model, and knockout the original reaction.

Parameters
----------
reaction: cobra.Reaction
    The reaction to swap.
model: cameo.cobra.Model
    A constraint-based model.
swap_pairs: tuple
    A tuple of (cofactors, equivalent_cofactors)
inplace: bool
    If replace is done inplace, it changes the coefficients in the matrix. Otherwise, it creates a new reaction
    with the other cofactors and adds it to the model.

Returns
-------
    Reaction
        A reaction with swapped cofactors (the same if inplace).
File:      ~/.local/lib/python3.6/site-packages/cameo/core/manipulation.py
Type:      function


In [179]:
?ReactionCofactorSwapTarget

Init signature: ReactionCofactorSwapTarget(id, swap_pairs, *args, **kwargs)
Docstring:      Swap cofactors of a given reaction.
File:           ~/.local/lib/python3.6/site-packages/cameo/core/target.py
Type:           type
Subclasses:     


In [181]:
?CofactorSwapOptimization

Init signature:
CofactorSwapOptimization(
    cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)
Docstring:     
Optimize co-factor swapping

As suggested in [1]_, flux through a given reaction can sometimes be optimized by swapping complementary
co-factor. This class implements a search for reactions when swapped improve the given objective. Briefly,
the approach is to

- find reactions that have all the targeted co-factor pairs e.g. (nad_c -> nadp_c, nadh_c -> nadph_c)

- add reactions that have the co-factors swapped and then by a search algorithm switching one off in favor of the
  other

The implementation here differs from that in [1]_ in that we use a general purpose search algorithm rather than
formulating the search as a mixed integer linear programming problem.

References
----------
.. [1] King, Zachary A., and Adam M. Feist. "Optimizing Cofactor Specificity of Oxidoreductase Enzym

In [205]:
swap_optimization = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True)

KeyError: 'nad_c'

In [200]:
NADH_NADPH = (['NAD', 'NADH'],['NADP', 'NADPH'])

In [201]:
print(NADH_NADPH)

(['NAD', 'NADH'], ['NADP', 'NADPH'])


In [ ]:
cofactor_id_swaps = NADH_NADPH

In [214]:
cofactor_id_swaps

(['NAD', 'NADH'], ['NADP', 'NADPH'])

In [217]:
cofactor_id_swaps = ([from 'nad_c'], [to 'NAD'])

SyntaxError: invalid syntax (<ipython-input-217-20a4bc197308>, line 1)

In [216]:
swap_optimization2 = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps)

SyntaxError: positional argument follows keyword argument (<ipython-input-216-4876dd87bd48>, line 1)

In [ ]:
CofactorSwapOptimization(
    cofactor_id_swaps=(['NAD', 'NADH'], ['NADP', 'NADPH']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)

Do we even have any cofactors?

PsiD: pyridoxal-5′-phosphate (PLP) i planter, men hvad med svampe?

PSIH: er en cytochrome P450 (so guess it use heme as cofactor?)
PcCpr: er en cp450 reductase som har cofaktorerne (FMN, FAD and NADPH)


PsiK: ATP 

PsiM x2: S-adenosyl-L-methionine (SAM) 

Skal ikke tilføje cofaktorerne i reaktionenerne! 


situationen med 2 enzymer i reaktion 2 forklaret:
så rekation 2 består altså både af  cytochrome P450 gene PcpsiH OG en cytochrome P450 reductase (CPR) partner 

"While psilocybin was successfully produced in yeast, the initial ti- ters were low. Furthermore, analysis revealed the extracellular accu- mulation of tryptamine (120.3 ± 11.1 mg/L) indicating a significant limitation in the conversion of tryptamine to 4-hydroxytryptamine encoded by the cytochrome P450 gene PcpsiH. Cytochrome P450 enzymes (CYP) belong to a superfamily of heme-containing mono- oxygenases and require a cytochrome P450 reductase (CPR) partner to deliver one or more electrons to reduce the heme-bound iron and oxi- dized substrates (Renault et al., 2014). While the detection of psilocybin indicated that the native S. cerevisiae CPR (Ncp1) could carry out this reduction, the accumulation of tryptamine suggested a sub-optimal"

"In this work, the successful production of psilocybin was demonstrated in Saccharomyces cerevisiae. Implementation of the biosynthetic pathway from Psilocybe cubensis with expression of a novel cytochrome P450 reductase resulted in high-level de novo production from glucose." ?